In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import csv
import cv2
import random

from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.models as models
from collections import OrderedDict
from torch.utils.data import Dataset, DataLoader

In [13]:
# !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !sudo apt-get update -qq 2>&1 > /dev/null
# !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
# !google-drive-ocamlfuse

In [12]:
# !sudo apt-get install -qq w3m # to act as web browser 
# !xdg-settings set default-web-browser w3m.desktop # to set default browser
# %cd /content
# !mkdir drive
# %cd drive
# !mkdir MyDrive
# %cd ..
# %cd ..
# !google-drive-ocamlfuse /content/drive/MyDrive

/content
mkdir: cannot create directory ‘drive’: File exists
/content/drive
mkdir: cannot create directory ‘MyDrive’: File exists
/content
/
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

MessageError: ignored

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle competitions download -c captcha-hacker
!unzip captcha-hacker.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/task2/CiRRerGOUkeBAQCZ.png  
  inflating: train/task2/CisfFJiwBkCP7ZJc.png  
  inflating: train/task2/Ciu3hIBZEkMS72b1.png  
  inflating: train/task2/CjLeyGNQ1F5EFJno.png  
  inflating: train/task2/CkpjVyGfldkertEP.png  
  inflating: train/task2/ClmmT4me3oVjU2uo.png  
  inflating: train/task2/CnWC54NdezW58Wiu.png  
  inflating: train/task2/CnuOrI1foWT8AIMh.png  
  inflating: train/task2/Cp6qg9AtP1oWIyxO.png  
  inflating: train/task2/Ct9G7oWGZs8EkjHN.png  
  inflating: train/task2/CzFBYB8r8vaCXGWy.png  
  inflating: train/task2/D0qdbF25TSdSjl8Q.png  
  inflating: train/task2/D1J6xGwPyit2XH7h.png  
  inflating: train/task2/D1LY7MfO4QnFQPKY.png  
  inflating: train/task2/D1W5bh1Wk3l5PLC4.png  
  inflating: train/task2/D1uDpwsCi15rDIrR.png  
  inflating: train/task2/D2EPn2DdXmdH3iUO.png  
  inflating: train/task2/D5Ece6l257nF0KKt.png  
  inflating: train/task2/D8RyGM7pKWCQuhO6.png  
  inflating: train/task2/DAyjJQTCgSDg

In [ ]:
CAPTCHA_DICT = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                'a', 'c', 'd', 'e', 'f', 'h','j', 'k', 'm', 'n', 'p', 'r', 's', 't','v','w','x','y' ]
TEST_PATH = "/content/test"
model1_path = "/content/model1.pth"
model2_path = "/content/model2.pth"
model3_path = "/content/model3.pth"
device = "cuda"


In [ ]:
def encoding(label):
    onehot_code = np.zeros(len(CAPTCHA_DICT) * len(label), dtype=float)
    for i, char in enumerate(label):
        index = i * len(CAPTCHA_DICT) + CAPTCHA_DICT.index(char)
        onehot_code[index] = 1.0
    return onehot_code
def decoding(onehot):
    p = ''
    for i, id in enumerate(np.where(onehot == 1)[0]):
        p += CAPTCHA_DICT[id - i*len(CAPTCHA_DICT)]
    return p
def decodingP(idx):
    p = ''
    for x in idx:
      p += CAPTCHA_DICT[x]
    return p

In [ ]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.medianBlur(img, 3)
        img = np.mean(img, axis=2)
        img = torch.FloatTensor((img - 128) / 128)
        img = img.view(1, *img.size())
        if self.return_filename:
            return img, filename
        else:
            return img, int(label)

    def __len__(self):
        return len(self.data)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3, stride = 1, padding = 1,),# padding = (kernel_size-1)/2 
        nn.BatchNorm2d(16),
        nn.ReLU(),# (16, 72, 72)
        nn.MaxPool2d(kernel_size = 2)# (16, 36, 36)
        )
        self.conv2 = nn.Sequential(    
        nn.Conv2d(16, 32, 3, 1, 1),# (32, 36, 36)
        nn.BatchNorm2d(32),
        nn.ReLU(),# (32,36,36)
        nn.MaxPool2d(2)# (32, 18, 18)
        )
        self.conv3 = nn.Sequential(    
        nn.Conv2d(32, 64, 3, 1, 1),# (64, 18, 18)
        nn.BatchNorm2d(64),
        nn.ReLU(),# (64,18,18)
        nn.MaxPool2d(2)# (64, 9, 9)
        )
        self.conv4 = nn.Sequential(    
        nn.Conv2d(64, 128, 3, 1, 1),# (128, 9, 9)
        nn.BatchNorm2d(128),
        nn.ReLU(),# (128,9,9)
        # nn.MaxPool2d(2)# (64, 4, 4)
        )
        self.out = nn.Sequential(
          nn.Linear(128*9*9, 1024),
          nn.ReLU(),
          nn.Dropout(),
          nn.Linear(1024, 256),
          nn.ReLU(),
          nn.Dropout(),
          nn.Linear( 256, 10)
        )
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0),-1)
        return self.out(x)

In [ ]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, return_filename=0):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (108, 108), interpolation=cv2.INTER_LANCZOS4)
        img = cv2.medianBlur(img, 3)
        img = np.mean(img, axis=2)
        img = torch.FloatTensor((img - 128) / 128)
        img = img.view(1, *img.size())
        if self.return_filename == 2:
            return img, filename
        elif self.return_filename == 1:
            return img, label
        else:
            return img, encoding(label)

    def __len__(self):
        return len(self.data)

In [ ]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, return_filename=0):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (144, 108), interpolation=cv2.INTER_LANCZOS4)
        img = np.mean(img, axis=2)
        img = torch.FloatTensor((img - 128) / 128)
        img = img.view(1, *img.size())
        if self.return_filename == 2:
            return img, filename
        elif self.return_filename == 1:
            return img, label
        else:
            return img, encoding(label)

    def __len__(self):
        return len(self.data)

In [ ]:

test_data = []
with open(f'/content/sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

test_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=500, num_workers=1, drop_last=False, shuffle=False)
print(len(test_ds))

if os.path.exists('submission.csv'):
    csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
else:
    csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
    csv_writer.writerow(["filename", "label"])
cnt = 0
model = Model().to(device)
model.load_state_dict(torch.load(model1_path))
model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    
    pred = model(image)
    pred = torch.argmax(pred, dim=1)
    
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], str(pred[i].item())])
        cnt += 1
test_ds = Task2Dataset(test_data, root=TEST_PATH, return_filename=2)
print(cnt)
print(len(test_ds))
model2 = models.resnet18(pretrained=True)
fc_features = model2.fc.in_features
model2.fc = nn.Linear(fc_features, len(CAPTCHA_DICT)*2)
model2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model2 = model2.to(device)
model2.load_state_dict(torch.load(model2_path))
test_dl = DataLoader(test_ds, batch_size=500, num_workers=1, drop_last=False, shuffle=False)
model2.eval()
for image, filenames in test_dl:
    image = image.to(device)
    pred = model2(image)
    pred = pred.view((pred.shape[0], 2, 28))
    pred = torch.argmax(pred, axis=2)
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], decodingP(pred[i].cpu().numpy())])
        # csv_writer.writerow([filenames[i], 0])
        cnt += 1
test_ds = Task3Dataset(test_data, root=TEST_PATH, return_filename=2)
print(cnt)
print(len(test_ds))
test_dl = DataLoader(test_ds, batch_size=200, num_workers=0, drop_last=False, shuffle=False)
model3 = models.resnet18(pretrained=True)
fc_features = model3.fc.in_features
model3.fc = nn.Linear(fc_features, len(CAPTCHA_DICT)*4)
model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3 = model3.to(device)
model3.load_state_dict(torch.load(model3_path))
model3.eval()
for image, filenames in test_dl:
    image = image.to(device)
    pred = model3(image)
    pred = pred.view((pred.shape[0], 4, 28))
    pred = torch.argmax(pred, axis=2)
    for i in range(len(filenames)):
        # csv_writer.writerow([filenames[i], 0])
        csv_writer.writerow([filenames[i], decodingP(pred[i].cpu().numpy())])
        cnt += 1
print(cnt)
cnt = 0
with open(f'/content/submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        cnt+=1
    print(cnt)
cnt = 10001 - cnt

csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
for i, (image, filenames) in enumerate(test_dl):
    if i == 1:
        image = image.to(device)
        pred = model3(image)
        pred = pred.view((pred.shape[0], 4, 28))
        pred = torch.argmax(pred, axis=2)
        for i in range(500 - cnt, len(filenames)):
            print(i, filenames[i])
            csv_writer.writerow([filenames[i], decodingP(pred[i].cpu().numpy())])
cnt = 0
with open(f'/content/submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        cnt+=1
    print(cnt)



6500
6500
2500


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

9000
1000
10000
9922
10001
